In [1]:
# Import Libraries
# Numerical
import numpy as np
import pandas as pd

# Graphing
import matplotlib.pyplot as plt
import seaborn as sns

# Encoding
from sklearn.preprocessing import LabelEncoder

# Clustering & PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans 

### Pre-defined Functions

In [43]:
def eda_subplots(data_df, graph_type='pie'):
    # Define the layout of subplots
    # Determine the number of subplot rows/columns needed
    num_columns = len(data_df.columns)
    # For simplicity, creating a square layout or as close to square as possible
    n_cols = int(np.ceil(np.sqrt(num_columns)))
    n_rows = int(np.ceil(num_columns / n_cols))

    # Creating the figure and axes for subplots
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(7*n_rows, 3*n_cols))

    # Iterate over each column to plot
    for i, column in enumerate(data_df.columns):
        # Aggregate data
        data = data_df[column].value_counts()
    
        # Check if axes is a single-dimensional array
        if n_rows == 1 or n_cols == 1:
            ax = axes[i]
        else:
            # For a 2D array of subplots, index with row and column
            row, col = i // n_cols, i % n_cols
            ax = axes[row, col]
    
        # Plot chart on the correct axis
        if graph_type == 'pie':
            ax.pie(data, labels=data.index, autopct='%1.1f%%', startangle=90)
        elif graph_type=='bar':
            ax.bar(data.index, data)
        else:
            ax.plot(data.index, data)
        ax.set_title(f'Pie Chart for {column}')

    plt.tight_layout()
    plt.show()


## 1.0 Load Datasets

In [44]:
data_key = pd.read_csv("data/QUESTION_KEY.csv")
data_key.head()

,Question Order,Question,Survey Section,Question Type,Answer Choices,Notes
0,1,What is your age?,Background Information,Single select,"<18 years old, 18-24, 25-34, 35-44, 45-54, 55-...",NaN
1,2,How many cups of coffee do you typically drink...,Background Information,Single select,"Less than 1, 1, 2, 3, 4, More than 4",NaN
2,3,Where do you typically drink coffee?,Background Information,Multiple selection,"None of these, At home, At a cafe, On the go, ...",NaN
3,4,"On the go, where do you typically purchase cof...",Background Information,Multiple selection,"Other, Drive-thru, Specialty coffee shop, Nati...","Only available if ""On the go"" was selected in ..."
4,5,Where else do you purchase coffee?,Background Information,Text,NaN,"Only available if ""Other"" was selected in ques..."


In [45]:
data = pd.read_csv("data/GACTT_RESULTS_ANONYMIZED_v2.csv", sep=',')
data.head()

,Submission ID,What is your age?,How many cups of coffee do you typically drink per day?,Where do you typically drink coffee?,Where do you typically drink coffee? (At home),Where do you typically drink coffee? (At the office),Where do you typically drink coffee? (On the go),Where do you typically drink coffee? (At a cafe),Where do you typically drink coffee? (None of these),How do you brew coffee at home?,...,What is the most you'd ever be willing to pay for a cup of coffee?,Do you feel like you’re getting good value for your money when you buy coffee at a cafe?,Approximately how much have you spent on coffee equipment in the past 5 years?,Do you feel like you’re getting good value for your money with regards to your coffee equipment?,Gender,Education Level,Ethnicity/Race,Employment Status,Number of Children,Political Affiliation
0,gMR29l,18-24 years old,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BkPN0e,25-34 years old,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pod/capsule machine (e.g. Keurig/Nespresso),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,W5G8jj,25-34 years old,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bean-to-cup machine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4xWgGr,35-44 years old,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coffee brewing machine (e.g. Mr. Coffee),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,QD27Q8,25-34 years old,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pour over,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.0 Data Exploration

In [46]:
data_key.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Question Order  56 non-null     int64 
 1   Question        56 non-null     object
 2   Survey Section  56 non-null     object
 3   Question Type   56 non-null     object
 4   Answer Choices  46 non-null     object
 5   Notes           10 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.8+ KB


In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4042 entries, 0 to 4041
Columns: 111 entries, Submission ID to Political Affiliation
dtypes: float64(21), object(90)
memory usage: 3.4+ MB


Separate data into participant deomgraphics, basic survey info, and coffee related info

In [48]:
desc_columns = [0, 1, 2, 89, 105, 106, 107, 108, 109, 110]
desc_df = data.iloc[:, desc_columns]
survey_columns = [20, 29, 65, 66, 67, 68, 69, 90, 98, 99, 100, 101, 102, 103, 104]
survey_df = data.iloc[:, survey_columns]
survey_enc_columns = [3,9,21,31, 38, 48,57,91]
coffee_survey_df = data.iloc[:, survey_columns + survey_enc_columns]
ds_data = data.iloc[:, desc_columns + survey_columns + survey_enc_columns]

### 2.1A Participant Descriptive Information

In [49]:
ds_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4042 entries, 0 to 4041
Data columns (total 33 columns):
 #   Column                                                                                            Non-Null Count  Dtype  
---  ------                                                                                            --------------  -----  
 0   Submission ID                                                                                     4042 non-null   object 
 1   What is your age?                                                                                 4011 non-null   object 
 2   How many cups of coffee do you typically drink per day?                                           3949 non-null   object 
 3   Do you work from home or in person?                                                               3524 non-null   object 
 4   Gender                                                                                            3523 non-null   object 
 5  

In [50]:
#print(ds_data.columns)
renamed_columns = ['ID', 'Age', 'DailyCoffeeCups', 'WorkMode', 'Gender', 'HighestEducation', 'Race', 'Employment', 'Children', 'Political',
    'HomeBrewCoffee','FavoriteCoffeeDrink', 'Pre_PreferredCoffee', 'CoffeeStrength', 'RoastLevel', 'CaffeineAmount', 'CoffeeExpertise',
    'MonthlyCoffeeSpend', 'LikeCoffeeTaste', 'KnowWhereCoffeeFrom', 'HighestPaidCoffeeCup', 'Willing_HighestPaidCoffeeCup','CafeCoffeeCup',
    'CoffeeEquipmentSpend', 'EquipmentCoffeeCup','CoffeeDrinkLocation', 'HowHomeCoffeeBrew', 'PurchaseCoffee','AddToCoffee','Dairy', 'SugarSweetner','Flavorings','WhyDrinkCoffee']
new_orig_dict = {}
for i in range(0,32):
    print(f"{renamed_columns[i]} - {ds_data.columns[i]}")
    new_orig_dict[renamed_columns[i]] = ds_data.columns[i]

ID - Submission ID
Age - What is your age?
DailyCoffeeCups - How many cups of coffee do you typically drink per day?
WorkMode - Do you work from home or in person?
Gender - Gender
HighestEducation - Education Level
Race - Ethnicity/Race
Employment - Employment Status
Children - Number of Children
Political - Political Affiliation
HomeBrewCoffee - How else do you brew coffee at home?
FavoriteCoffeeDrink - What is your favorite coffee drink?
Pre_PreferredCoffee - Before today's tasting, which of the following best described what kind of coffee you like?
CoffeeStrength - How strong do you like your coffee?
RoastLevel - What roast level of coffee do you prefer?
CaffeineAmount - How much caffeine do you like in your coffee?
CoffeeExpertise - Lastly, how would you rate your own coffee expertise?
MonthlyCoffeeSpend - In total, much money do you typically spend on coffee in a month?
LikeCoffeeTaste - Do you like the taste of coffee?
KnowWhereCoffeeFrom - Do you know where your coffee comes fro

In [51]:
# Rename columns
ds_data.columns = renamed_columns

In [52]:
ds_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4042 entries, 0 to 4041
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            4042 non-null   object 
 1   Age                           4011 non-null   object 
 2   DailyCoffeeCups               3949 non-null   object 
 3   WorkMode                      3524 non-null   object 
 4   Gender                        3523 non-null   object 
 5   HighestEducation              3438 non-null   object 
 6   Race                          3418 non-null   object 
 7   Employment                    3419 non-null   object 
 8   Children                      3406 non-null   object 
 9   Political                     3289 non-null   object 
 10  HomeBrewCoffee                678 non-null    object 
 11  FavoriteCoffeeDrink           3980 non-null   object 
 12  Pre_PreferredCoffee           3958 non-null   object 
 13  Cof

In [53]:
ds_data.isna().sum().sort_values(ascending=False)

Flavorings                      4042
SugarSweetner                   3530
HomeBrewCoffee                  3364
PurchaseCoffee                  3332
Dairy                           2356
Political                        753
Children                         636
Race                             624
Employment                       623
HighestEducation                 604
EquipmentCoffeeCup               548
CafeCoffeeCup                    542
CoffeeEquipmentSpend             536
Willing_HighestPaidCoffeeCup     532
MonthlyCoffeeSpend               531
Gender                           519
WorkMode                         518
HighestPaidCoffeeCup             515
KnowWhereCoffeeFrom              483
LikeCoffeeTaste                  479
WhyDrinkCoffee                   474
HowHomeCoffeeBrew                385
CoffeeStrength                   126
CaffeineAmount                   125
CoffeeExpertise                  104
RoastLevel                       102
DailyCoffeeCups                   93
P

In [64]:
highest_null_columns = ds_data.isna().sum().sort_values(ascending=False).head().index
ds_data[highest_null_columns].nunique()

Flavorings          0
SugarSweetner      82
HomeBrewCoffee    170
PurchaseCoffee     89
Dairy             175
dtype: int64

In [81]:
# Drop columns with the five highest null values of over half the row size
ds_data.drop(columns=highest_null_columns, inplace=True)

C:\Users\kymno\AppData\Local\Temp\ipykernel_28400\1889198058.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_data.drop(columns=highest_null_columns[1:], inplace=True)


### Handling the Nulls
Decide for these values if we need to impute nulls for some columns and/or drop the rows with nulls

### 2.1A Data Cleaning

- Deal with Missing Values 
- Checking for Duplicates
- changing object formatted columns to Int


In [82]:
ds_data.isna().sum().sort_values(ascending=False)

Political                       753
Children                        636
Race                            624
Employment                      623
HighestEducation                604
EquipmentCoffeeCup              548
CafeCoffeeCup                   542
CoffeeEquipmentSpend            536
Willing_HighestPaidCoffeeCup    532
MonthlyCoffeeSpend              531
Gender                          519
WorkMode                        518
HighestPaidCoffeeCup            515
KnowWhereCoffeeFrom             483
LikeCoffeeTaste                 479
WhyDrinkCoffee                  474
HowHomeCoffeeBrew               385
CoffeeStrength                  126
CaffeineAmount                  125
CoffeeExpertise                 104
RoastLevel                      102
DailyCoffeeCups                  93
Pre_PreferredCoffee              84
AddToCoffee                      83
CoffeeDrinkLocation              70
FavoriteCoffeeDrink              62
Age                              31
ID                          

In [84]:
# The Political column has the most nulls
ds_data['Political'].value_counts()

Democrat          1768
No affiliation     826
Independent        507
Republican         188
Name: Political, dtype: int64

In [85]:
# Impute with none given
ds_data['Political'].fillna('Not Specified', inplace=True)

C:\Users\kymno\AppData\Local\Temp\ipykernel_28400\3271807493.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_data['Political'].fillna('Not Specified', inplace=True)


Since not having any children is not listed in the value counts, then it makes sense to replace the null values in the column with 0.  

In [86]:
ds_data['Children'].value_counts()

None           2550
2               402
1               310
3                92
More than 3      52
Name: Children, dtype: int64

In [87]:
# Impute the blanks with 0 and replace None with 0
ds_data['Children'].fillna(0, inplace=True)
ds_data['Children'].replace('None', 0, inplace=True)

C:\Users\kymno\AppData\Local\Temp\ipykernel_28400\1531306829.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_data['Children'].fillna(0, inplace=True)
C:\Users\kymno\AppData\Local\Temp\ipykernel_28400\1531306829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_data['Children'].replace('None', 0, inplace=True)


In [ ]:
d_df = desc_df.dropna().reset_index()
d_df.info()

In [ ]:
# Checking for duplicates
d_df.duplicated().sum()

In [ ]:
d_df['Gender'].value_counts()

In [ ]:
pie_subplots(d_df)

Since each column has between 3 and 7 different entry options, I will apply a label encoding for each of the columns


In [ ]:
d_df.iloc[:,1].value_counts()

In [ ]:
le = LabelEncoder()
d_df['le_Age'] = le.fit_transform(d_df.iloc[:,1])
d_df['le_Age'].value_counts()

In [ ]:
test_age = le.inverse_transform(d_df['Age'])
test_age

In [ ]:
d_df.columns

In [ ]:
d_df.iloc[:,2].value_counts()

In [ ]:
d_df['CoffeeCups'] = d_df.iloc[:,2].map({"Less than 1":0, "1":1, "2":2, "3":3, "4":4, "More than 4":5})
d_df['CoffeeCups'].value_counts()

In [ ]:
d_df.iloc[:,3].value_counts()

In [ ]:
d_df['WorkLocation'] = le.fit_transform(d_df.iloc[:,3])
d_df['WorkLocation'].value_counts()

In [ ]:
d_df['Gender'].value_counts()

In [ ]:
d_df['le_Gender'] = le.fit_transform(d_df.iloc[:,4])
d_df['le_Gender'].value_counts()

In [ ]:
d_df['Education Level'].value_counts()

In [ ]:
d_df['le_HighestEducation'] = le.fit_transform(d_df.iloc[:,5])
d_df['le_HighestEducation'].value_counts()

In [ ]:
d_df['Ethnicity/Race'].value_counts()

In [ ]:
d_df['le_Race']=le.fit_transform(d_df.iloc[:,6])
d_df['le_Race'].value_counts()

In [ ]:
d_df['Employment Status'].value_counts()

In [ ]:
d_df['le_Employment'] = le.fit_transform(d_df.iloc[:,7])
d_df['le_Employment'].value_counts()

In [ ]:
d_df['Children'] = d_df.iloc[:,8].map({"0":0, "1":1, "2":2, "3":3, "More than 3":4})
d_df['Children'].value_counts()

In [ ]:
d_df['Children'] = d_df['Children'].astype(int)

In [ ]:
d_df['Political Affiliation'].value_counts()

In [ ]:
d_df['PoliticalParty']=le.fit_transform(d_df.iloc[:, 9])
d_df['PoliticalParty'].value_counts()

In [ ]:
d_df.info()

In [ ]:
df = d_df.iloc[:, 10:]

## 2.1B Survey Results

In [ ]:
survey_df.info()

In [ ]:
s_df=survey_df.iloc[:,1:].dropna()
s_df.info()

In [ ]:
s_df.iloc[:, 0].value_counts()

In [ ]:
pie_subplots(s_df)

In [ ]:
s_df['FavoriteDrink']=le.fit_transform(s_df.iloc[:, 0])
s_df['FavoriteDrink'].value_counts()

In [ ]:
s_df.iloc[:, 1].value_counts()

In [ ]:
s_df['Pre_PreferredCoffee']=le.fit_transform(s_df.iloc[:, 1])
s_df['Pre_PreferredCoffee'].value_counts()

In [ ]:
s_df.iloc[:, 2].value_counts()

In [ ]:
s_df['CoffeeStrength']=le.fit_transform(s_df.iloc[:, 2])
s_df['CoffeeStrength'].value_counts()

In [ ]:
s_df.iloc[:, 3].value_counts()

In [ ]:
s_df['CoffeeRoast']=le.fit_transform(s_df.iloc[:, 3])
s_df['CoffeeRoast'].value_counts()

In [ ]:
s_df.iloc[:, 4].value_counts()

In [ ]:
s_df['CaffeineLevel']=le.fit_transform(s_df.iloc[:, 4])
s_df['CaffeineLevel'].value_counts()

In [ ]:
s_df.iloc[:, 6].value_counts()

In [ ]:
s_df['CoffeeSpend']=le.fit_transform(s_df.iloc[:, 6])
s_df['CoffeeSpend'].value_counts()

In [ ]:
s_df.iloc[:, 9].value_counts()

In [ ]:
s_df['HighestCoffeeSpend']=le.fit_transform(s_df.iloc[:, 9])
s_df['HighestCoffeeSpend'].value_counts()

In [ ]:
s_df.iloc[:, 10].value_counts()

In [ ]:
s_df['Willing_HighestCoffeeSpend']=le.fit_transform(s_df.iloc[:, 10])
s_df['Willing_HighestCoffeeSpend'].value_counts()

In [ ]:
s_df.iloc[:, 12].value_counts()

In [ ]:
s_df['CoffeeEquipmentSpend']=le.fit_transform(s_df.iloc[:, 12])
s_df['CoffeeEquipmentSpend'].value_counts()

In [ ]:
# Convert Yes No questions into 0s and 1s
yes_no_cols = [7,8,11,13]
yes_no_new_names = ["CoffeeTest","CoffeeFrom", "CafeCoffeeValue", "EquipCoffeeValue"]
for i in range(0, len(yes_no_cols)):
    s_df[yes_no_new_names[i]] = s_df.iloc[:,yes_no_cols[i]].map({"No":0, "Yes":1})

In [ ]:
s_df[yes_no_new_names].describe().T

### Convert the Object columns to Int columns using the Label Encoder

In [ ]:
pie_subplots(s_df.iloc[:,1:])

In [ ]:
s_df['CoffeeExpertiseRating']=s_df.iloc[:,5]
s_df.iloc[:,14:].info()

In [ ]:
df_2 = s_df.iloc[:,14:]

In [ ]:
survey_corr = df_2.corr()
fig, ax = plt.subplots(figsize=(15,10)) 
sns.heatmap(df_2.corr())

In [ ]:
survey_corr